In [1]:
import os
import pandas as pd
from scipy import stats as st
import numpy as np
os.chdir("C:\\Users\\manu")
data = pd.read_csv("Attrition.csv")

In [2]:
data.head(1)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5


In [3]:
data.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

#### No missing value in the data

In [4]:
data["Attrition"].value_counts()#imbalanced data

No     1233
Yes     237
Name: Attrition, dtype: int64

#### data is imbalanced, stratified sampling for train test division

In [70]:
import EDA
import feature_engeneering as fe
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split

#### for EDA, feature engineering function link: https://goo.gl/THdGHu

In [71]:
EDA.Graph(data,"F:\\attri proj") #distribution of variables through graphs

#### some of the variables are redundent for analysis which is more or less constant and ids, and outliers are present in few of the variables and some variables are skewed as well

In [72]:
fe.feature_select(data,1,0.01) #Selecting feature on the basis of t-test and chi-square

C:\Users\manu\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\morestats.py:1306: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
C:\Users\manu\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\manu\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\manu\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


['Age',
 'BusinessTravel',
 'Department',
 'DistanceFromHome',
 'EducationField',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'OverTime',
 'StockOptionLevel',
 'TotalWorkingYears',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsWithCurrManager']

#### by feature function, redundent variable that does not have any impact due to attrition are droped out. method for droppin:
 #### chi square test for catagorical, and t test for numerical

In [75]:
X = data[['Age', 'BusinessTravel', 'Department', 'DistanceFromHome', 
              'EducationField', 'EnvironmentSatisfaction', 'JobInvolvement', 'JobLevel', 'JobRole',
               'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'OverTime', 'StockOptionLevel', 'TotalWorkingYears',
              'YearsAtCompany', 'YearsInCurrentRole', 'YearsWithCurrManager']] #subsetting for new features
Y = data[["Attrition"]]

In [76]:
EDA.Graph(X,"F:\\attri proj") #distribution of subset data

In [77]:
X.dtypes

Age                         int64
BusinessTravel             object
Department                 object
DistanceFromHome            int64
EducationField             object
EnvironmentSatisfaction     int64
JobInvolvement              int64
JobLevel                    int64
JobRole                    object
JobSatisfaction             int64
MaritalStatus              object
MonthlyIncome               int64
OverTime                   object
StockOptionLevel            int64
TotalWorkingYears           int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsWithCurrManager        int64
dtype: object

In [78]:
##label encoding

le = LabelEncoder()
y = le.fit_transform(Y)

C:\Users\manu\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [79]:
x=X[['BusinessTravel', 'Department','EducationField','JobRole','MaritalStatus','OverTime']].apply(LabelEncoder().fit_transform)

In [80]:
X1=X.drop(['BusinessTravel', 'Department','EducationField','JobRole','MaritalStatus','OverTime'],axis=1)

In [81]:
dat = pd.concat([X1,x],axis=1)

## Test train split using stratified random sampling

In [82]:
x_train, x_test, y_train, y_test = train_test_split(dat,y,test_size = 0.2,stratify=y,random_state=50)

#### importing necessary functions for Feature engeneering using Recursive feature elimination

In [87]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [88]:
model = DecisionTreeClassifier(criterion="gini",max_depth=7,min_samples_split=.05)#using decision tree in RFE for feature selection

In [100]:
mod=RFE(model,step = 300)

In [101]:
fitt=mod.fit(x_train,y_train)

In [190]:
lis=fitt.support_

In [145]:
colmn=[] ## lis has binary output, use of for loop to extract out column names
feature=x_train.columns
for i in range(len(lis)):
    if lis[i]==True:
        colmn.append(feature[i])#filling column names in colmn
        

In [146]:
colmn

['DistanceFromHome',
 'JobInvolvement',
 'MonthlyIncome',
 'StockOptionLevel',
 'TotalWorkingYears',
 'YearsAtCompany',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [152]:
X_train = x_train[colmn]#subsetting train and test using colmn
X_test = x_test[colmn]

In [183]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=7, min_samples_split=.05)
clf_gini.fit(X_train, y_train)#training model on train data

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=0.05,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

### Checking accuracy of model using confusion matrix

In [184]:
from sklearn.metrics import classification_report, confusion_matrix 

In [185]:
y_pred = clf_gini.predict(X_test)#predicting the outcome using test data

In [186]:
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

[[242   5]
 [ 34  13]]
             precision    recall  f1-score   support

          0       0.88      0.98      0.93       247
          1       0.72      0.28      0.40        47

avg / total       0.85      0.87      0.84       294



In [191]:
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.1326530612244898
Mean Squared Error: 0.1326530612244898
Root Mean Squared Error: 0.3642156795423418
